<a href="https://colab.research.google.com/github/roseandgold/HatefulMemesProject/blob/main/BERT%20Model/BERT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Model
Some code was adapted from Coursera guided project Sentiment Analysis with BERT

Best if run in Colab using GPU

## Step 1: Load the Data

In [1]:
%%capture
!pip install transformers

In [2]:
import torch
import random
import pandas as pd
import numpy as np
import plotly.express as px
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup

In [3]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# Import necessary files
train_file = '/content/drive/MyDrive/SIADS 697 - Capstone/SIADS 697 - Capstone/hateful_memes/train.jsonl'
test_file = '/content/drive/MyDrive/SIADS 697 - Capstone/SIADS 697 - Capstone/hateful_memes/dev_seen.jsonl'
final_test_file = '/content/drive/MyDrive/SIADS 697 - Capstone/SIADS 697 - Capstone/hateful_memes/test_seen.jsonl'

train_data = pd.read_json(train_file, lines = True)
test_data = pd.read_json(test_file, lines = True)
final_test_data = pd.read_json(final_test_file, lines = True)
test_data.head()

,id,img,label,text
0,8291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...
4,80243,img/80243.png,1,mississippi wind chime


## Step 2: Loading Tokenizer and Encoding our Data

In [5]:
# Create the tokenizer
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
# Adapted from Coursera guided project "Sentiment Analysis with BERT"
encoded_data_train = tokenizer.batch_encode_plus(
    train_data.text.values,
    add_special_tokens=True, # set this so BERT knows when a sentence ends and begins
    return_attention_mask=True, #using a fixed input, all sentences have same dimensionality. Tells us where actual values are and where zeros are
    padding = True, # make sure to pad sentences so they are all the same length
    max_length = 256, # this is the length we want all senteces to be
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    test_data.text.values,
    add_special_tokens=True, # set this so BERT knows when a sentence ends and begins
    return_attention_mask=True, #using a fixed input, all sentences have same dimensionality. Tells us where actual values are and where zeros are
    padding = True, # make sure to pad sentences so they are all the same length
    max_length = 256, # this is the length we want all senteces to be
    return_tensors = 'pt'
)

encoded_final_test= tokenizer.batch_encode_plus(
    final_test_data.text.values,
    add_special_tokens=True, # set this so BERT knows when a sentence ends and begins
    return_attention_mask=True, #using a fixed input, all sentences have same dimensionality. Tells us where actual values are and where zeros are
    padding = True, # make sure to pad sentences so they are all the same length
    max_length = 256, # this is the length we want all senteces to be
    return_tensors = 'pt'
)

# get the parts from the encoding that need to train the model
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_data.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(test_data.label.values)

input_ids_final_test = encoded_final_test['input_ids']
attention_masks_final_test = encoded_final_test['attention_mask']
labels_final_test = torch.tensor(final_test_data.label.values)

In [7]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_final_test = TensorDataset(input_ids_final_test, attention_masks_final_test, labels_final_test)

In [8]:
print(len(dataset_train))
print(len(dataset_val))
print(len(dataset_final_test))

8500
500
1000


## Step 3: Setting up BERT Pretrained Model

In [9]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels=len(train_data.label.unique()),
    output_attentions=False,
    output_hidden_states=False
)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Step 4: Creating Data Loaders

In [10]:
# Adapted from Coursera guided project "Sentiment Analysis with BERT"
batch_size=32

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

dataloader_final_test = DataLoader(
    dataset_final_test,
    sampler=RandomSampler(dataset_final_test),
    batch_size=32
)

## Step 5: Setting Up Optimizer and Scheduler

In [11]:
# Adam is a stochastic optimization
optimizer = AdamW(
    model.parameters(),
    lr= 1e-5, #2e-5 > 5e -5
    eps=1e-8
)

In [12]:
epochs = 30

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train) *epochs
)

## Step 6: Defining our Performance Metric

In [13]:
def auc(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return roc_auc_score(labels_flat, preds_flat)

## Step 7: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [14]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val) #usually if using GPU

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [16]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [17]:
for epoch in tqdm(range(1, epochs+1)):
    model.train() # put model in training mode
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                       leave = False,
                       disable = False)
    for batch in progress_bar:
        
        model.zero_grad() # set the gradient to zero
        
        batch = tuple(b.to(device) for b in batch) # make sure each item on device
        
        # get the inputs to the model
        inputs = {
            'input_ids': batch[0],
            'attention_mask' : batch[1],
            'labels': batch[2]
        }
    
        # get the outputs
        outputs = model(**inputs)
        
        #BERT returns loss and logits
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward() # backpropagation
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # prevents gradient from getting too small or too big
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/SIADS 697 - Capstone/SIADS 697 - Capstone/hateful_memes/models/BERT2_ft_epoch{epoch}.model')
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_value = evaluate(dataloader_val)
    val_AUC = auc(predictions, true_value)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'AUC Score: {val_AUC}')

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/266 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.5719578149623441


  0%|          | 0/16 [00:00<?, ?it/s]

Validation loss: 0.7584730572998524
AUC Score: 0.5782192635739546


...

Validation loss: 2.6044616773724556
AUC Score: 0.5443823910643133


Epoch 30:   0%|          | 0/266 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.1415129067180188


  0%|          | 0/16 [00:00<?, ?it/s]

Validation loss: 2.5800046399235725
AUC Score: 0.5444784048903042


## Step 8: Loading and Evaluating our Model

In [17]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(train_data.label.unique()),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.to(device)
pass

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [45]:
# load a pretrained model
model.load_state_dict(
    torch.load('/content/drive/MyDrive/SIADS 697 - Capstone/SIADS 697 - Capstone/hateful_memes/models/BERT2_ft_epoch1.model',
    map_location = torch.device('cpu'))
)

<All keys matched successfully>

In [46]:
# Get the predictions for the hold out test set
_, predictions, true_vals = evaluate(dataloader_final_test)

  0%|          | 0/32 [00:00<?, ?it/s]

In [47]:
# Get the predictions for the training set
_, train_predictions, train_true_vals = evaluate(dataloader_train)

  0%|          | 0/266 [00:00<?, ?it/s]

In [48]:
# Get the AUC on the hold out test set
final_test_auc = auc(predictions, true_vals)
train_auc = auc(train_predictions, train_true_vals)
print(final_test_auc)
print(train_auc)

0.6019007603041217
0.7506777455607281


In [25]:
# Create the roc curve
def create_roc_curve(test_predictions, test_true_vals, train_predictions, train_true_vals, final_test_auc):
    test_fpr, test_tpr, _ = roc_curve(test_true_vals, test_predictions, pos_label = 1)
    train_fpr, train_tpr, _ = roc_curve(train_true_vals, train_predictions, pos_label = 1)
    
    # Create a dataframe of fpr and tpr to create visual
    testRocDf = pd.DataFrame({'fpr': test_fpr, 'tpr': test_tpr, 'data':['test']*len(test_tpr)})
    trainRocDf = pd.DataFrame({'fpr': train_fpr, 'tpr': train_tpr, 'data': ['train'] * len(train_tpr)})
    rocDf = pd.concat([testRocDf, trainRocDf])

    # Create the ROC curve
    fig = px.line(rocDf, 
                  x = 'fpr', 
                  y = 'tpr',
                  color = 'data',
                  color_discrete_sequence=['orange', 'blue'],
                 title = 'BERT ROC Curve (Test AUC: {})'.format(round(final_test_auc,2)))
    fig.update_layout({
        'plot_bgcolor': 'rgba(0,0,0,0)',
        'paper_bgcolor':'rgba(0,0,0,0)'
    })
    fig.show()
    return rocDf

In [26]:
rocDf = create_roc_curve(predictions[:,1], true_vals, train_predictions[:,1], train_true_vals, final_test_auc)

In [27]:
rocDf.head()

,fpr,tpr,data
0,0.0,0.000000,test
1,0.0,0.002041,test
2,0.0,0.006122,test
3,0.0,0.010204,test
4,0.0,0.012245,test
